In [1]:
# Import Required Modules
from flask import Flask, render_template
import pandas as pd
import json
import plotly
import plotly.express as px
import pandas_ta as pta
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import date
from datetime import datetime
from datetime import timedelta, date
from prophet import Prophet
import pandas as pd
import numpy as np
import yfinance as yf
import seaborn as sns
import mpld3
import matplotlib.pyplot as plt
import warnings

   
df = pd.read_csv("Resources/btcjoin.csv", parse_dates=['date'])
btc_df = yf.Ticker('BTC-USD').history(period='7y',interval='1d',actions=False).reset_index()
btc_df = btc_df.loc[(btc_df['Date'] > '2022-10-25')]
btc_df['Close']=btc_df['Close'].astype("float")
df['price']=df['price'].str.replace(',','')
df['price']=df['price'].astype("float")
btc_df = btc_df.rename(columns={"Close": "price", "Date":"date"})
btc_df['date'] = btc_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
btc_df['date'] = pd.to_datetime(btc_df['date'])

df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = df.rename(columns={"value": "wallets"})
df = df.drop(columns=['volume','change', 'low', 'high', 'open','Unnamed: 0', "wallets", "address", "mined"])

c:\Users\chris\anaconda3\envs\python_data\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [2]:
df = df.drop(columns=['Open_x','High_x', 'Low_x', 'Volume_x', 'Open_y','High_y', "Low_y", "Volume_y"])
df

,date,price
0,2010-09-25,0.100000
1,2010-09-26,0.100000
2,2010-09-27,0.100000
3,2010-09-28,0.100000
4,2010-09-29,0.100000
...,...,...
5168,2024-12-16,106029.718750
5169,2024-12-17,106140.601562
5170,2024-12-18,100041.539062
5171,2024-12-19,97490.953125


In [3]:
import numpy as np

df['log10 index'] = np.log10(df.index)
df['log10 price'] = np.log10(df['price'])
df

c:\Users\chris\anaconda3\envs\python_data\Lib\site-packages\pandas\core\indexes\base.py:945: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*new_inputs, **kwargs)


,date,price,log10 index,log10 price
0,2010-09-25,0.100000,-inf,-1.000000
1,2010-09-26,0.100000,0.000000,-1.000000
2,2010-09-27,0.100000,0.301030,-1.000000
3,2010-09-28,0.100000,0.477121,-1.000000
4,2010-09-29,0.100000,0.602060,-1.000000
...,...,...,...,...
5168,2024-12-16,106029.718750,3.713323,5.025428
5169,2024-12-17,106140.601562,3.713407,5.025882
5170,2024-12-18,100041.539062,3.713491,5.000180
5171,2024-12-19,97490.953125,3.713575,4.988964


In [4]:
import plotly.graph_objects as go

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add left y-axis
fig.add_trace(
    go.Scatter(x=df['date'], y=df['price'], name='Price', line=dict(color='blue')),
    secondary_y=False
)

# Add right y-axis
fig.add_trace(
    go.Scatter(x=df['date'], y=df['log10 price'], name='Log10 Price', line=dict(color='red')),
    secondary_y=True
)

# Set x-axis label
fig.update_xaxes(title_text='Date')

# Set left y-axis label
fig.update_yaxes(title_text='Price', secondary_y=False)

# Set right y-axis label
fig.update_yaxes(title_text='Log10 Price', secondary_y=True)
# Set left y-axis scale to log
fig.update_yaxes(type='log', secondary_y=False)
fig.update_layout(
    width=900,  # Set the width of the plot
    height=600  # Set the height of the plot
)

fig.show()

In [5]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add left y-axis
fig.add_trace(
    go.Scatter(x=df.index, y=df['price'], name='Price', line=dict(color='blue')),
    secondary_y=False
)

# Add right y-axis
fig.add_trace(
    go.Scatter(x=df.index, y=df['log10 price'], name='Log10 Price', line=dict(color='red')),
    secondary_y=True
)

# Set x-axis label
fig.update_xaxes(title_text='Index')

# Set left y-axis label
fig.update_yaxes(title_text='Price', secondary_y=False)

# Set right y-axis label
fig.update_yaxes(title_text='Log10 Price', secondary_y=True)
# Set left y-axis scale to log
fig.update_yaxes(type='log', secondary_y=False)
fig.update_layout(
    width=900,  # Set the width of the plot
    height=600  # Set the height of the plot
)

fig.show()

In [6]:
import numpy as np
import plotly.graph_objects as go

# Filter out rows with NaN or infinite values
filtered_df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=['log10 index', 'log10 price'])

# Further filter to include only rows where 'log10 index' >= 1.5
filtered_df = filtered_df[filtered_df['log10 index'] >= 1.5]

# Select the second row
row_index = 1
log10_price = filtered_df.iloc[row_index]['log10 price']
log10_index = filtered_df.iloc[row_index]['log10 index']

# Fit the power law regression line
coeffs = np.polyfit(filtered_df['log10 index'], filtered_df['log10 price'], 1)

# Generate the regression line
regression_line = coeffs[0] * filtered_df['log10 index'] + coeffs[1]

# Create the plot
fig = go.Figure()

# Add data points
fig.add_trace(go.Scatter(
    x=filtered_df['log10 index'],
    y=filtered_df['log10 price'],
    mode='markers',
    name='Data'
))

# Add regression line
fig.add_trace(go.Scatter(
    x=filtered_df['log10 index'],
    y=regression_line,
    mode='lines',
    name='Regression Line',
    line=dict(color='red')
))

# Add selected point
fig.add_trace(go.Scatter(
    x=[log10_index],
    y=[log10_price],
    mode='markers',
    name='Selected Point',
    marker=dict(color='green', size=10)
))

# Set axis labels
fig.update_layout(
    xaxis_title='Log10 Index',
    yaxis_title='Log10 Price',
    width=900,  # Set the width of the plot
    height=600  # Set the height of the plot
)

# Show the plot
fig.show()

In [7]:
from scipy.stats import linregress

# Calculate the regression line
slope, intercept, r_value, p_value, std_err = linregress(filtered_df['log10 index'], filtered_df['log10 price'])

# Display the equation and R value
equation = f'y = {slope:.2f}x + {intercept:.2f}'
r_value = f'R = {r_value:.2f}'

equation, r_value

('y = 3.36x + -7.94', 'R = 0.96')

In [8]:
import numpy as np

import plotly.graph_objects as go

# Generate the x values for prediction
x_pred = np.arange(1.704922, 5.704922, 0.1)
# ('y = 3.35x + -7.88', 'R = 0.95')
# Predict the y values using the equation
y_pred = coeffs[0] * x_pred + coeffs[1]

# Create the plot
fig = go.Figure()

# Add the predicted values
fig.add_trace(go.Scatter(
    x=x_pred,
    y=y_pred,
    mode='lines',
    name='Predicted Values',
    line=dict(color='green')
))

# Add the existing data points
fig.add_trace(go.Scatter(
    x=filtered_df['log10 index'],
    y=filtered_df['log10 price'],
    mode='markers',
    name='Data'
))

# Set axis labels
fig.update_layout(
    xaxis_title='Log10 Index',
    yaxis_title='Log10 Price',
    width=900,  # Set the width of the plot
    height=600  # Set the height of the plot
)

# Show the plot
fig.show()

In [9]:
import pandas as pd
import numpy as np

# Generate the x values for prediction
x_pred = np.arange(1.704922, 5.704922, 0.1)

# Predict the y values using the equation
y_pred = coeffs[0] * x_pred + coeffs[1]

# Create a DataFrame with x and y values
pred_df = pd.DataFrame({'x': x_pred, 'y': y_pred})
pred_df['index'] = 10 ** pred_df['x']
pred_df['price'] = 10 ** pred_df['y']
pred_df['price'] = pred_df['price'].apply(lambda x: "${:,.2f}".format(x))

pred_df['date'] = pred_df['index'].apply(lambda x: round(x, 1))
pred_df['date'] = pred_df['date'].astype(int)
pred_df


,x,y,index,price,date
0,1.704922,-2.199431,50.689966,$0.01,50
1,1.804922,-1.862938,63.814886,$0.01,63
2,1.904922,-1.526444,80.338182,$0.03,80
3,2.004922,-1.189951,101.139779,$0.06,101
4,2.104922,-0.853457,127.327438,$0.14,127
5,2.204922,-0.516964,160.295747,$0.30,160
6,2.304922,-0.180471,201.800389,$0.66,201
7,2.404922,0.156023,254.051638,$1.43,254
8,2.504922,0.492516,319.832063,$3.11,319
9,2.604922,0.829009,402.644712,$6.75,402


In [10]:
import pandas as pd
import numpy as np

# Generate the x values for prediction
x_pred = np.arange(1.704922, 4.2, 0.1)

# Predict the y values using the equation
y_pred = coeffs[0] * x_pred + coeffs[1]

# Create a DataFrame with x and y values
pred_df = pd.DataFrame({'x': x_pred, 'y': y_pred})
pred_df['index'] = 10 ** pred_df['x']
pred_df['price'] = 10 ** pred_df['y']
pred_df['price'] = pred_df['price'].apply(lambda x: "${:,.2f}".format(x))

# Round the index values and convert to integer
pred_df['date'] = pred_df['index'].apply(lambda x: round(x, 1))
pred_df['date'] = pred_df['date'].astype(int)

# Define the start date
start_date = pd.Timestamp('2010-09-25')

# Calculate the pred_date by adding the date values as days to the start date for the first two rows
pred_df.loc[:50, 'pred_date'] = pred_df.loc[:50, 'date'].apply(lambda x: (start_date + pd.to_timedelta(x, unit='D')).strftime('%Y-%m-%d'))
# Create a new column in pred_df to store the associated price from df

pred_df = pred_df[['price', 'pred_date','date']]

import pandas as pd
import numpy as np

# Assuming df and pred_df are already defined

# Convert the 'date' column in df to datetime
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

# Convert the 'pred_date' column in pred_df to datetime
pred_df['pred_date'] = pd.to_datetime(pred_df['pred_date'], format='%Y-%m-%d')

# Now merge the DataFrames
merged_df = pd.merge(df, pred_df, left_on='date', right_on='pred_date', how='inner')

merged_pred_df = pd.merge(merged_df, pred_df, on='pred_date', how='outer')
merged_pred_df

,date_x,price_x,log10 index,log10 price,price_y,pred_date,date_y,price,date
0,2010-11-14,0.300000,1.698970,-0.522879,$0.01,2010-11-14,50.0,$0.01,50
1,2010-11-27,0.300000,1.799341,-0.522879,$0.01,2010-11-27,63.0,$0.01,63
2,2010-12-14,0.200000,1.903090,-0.698970,$0.03,2010-12-14,80.0,$0.03,80
3,2011-01-04,0.300000,2.004321,-0.522879,$0.06,2011-01-04,101.0,$0.06,101
4,2011-01-30,0.500000,2.103804,-0.301030,$0.14,2011-01-30,127.0,$0.14,127
5,2011-03-04,0.900000,2.204120,-0.045757,$0.30,2011-03-04,160.0,$0.30,160
6,2011-04-14,1.000000,2.303196,0.000000,$0.66,2011-04-14,201.0,$0.66,201
7,2011-06-06,18.500000,2.404834,1.267172,$1.43,2011-06-06,254.0,$1.43,254
8,2011-08-10,10.000000,2.503791,1.000000,$3.11,2011-08-10,319.0,$3.11,319
9,2011-11-01,3.200000,2.604226,0.505150,$6.75,2011-11-01,402.0,$6.75,402


In [11]:
import plotly.graph_objects as go

# Create the plot
fig = go.Figure()

# Add the price_x data
fig.add_trace(go.Scatter(
    x=merged_pred_df['pred_date'],
    y=merged_pred_df['price_x'],
    mode='lines',
    name='Actual Price'
))

# Add the price_y data
fig.add_trace(go.Scatter(
    x=merged_pred_df['pred_date'],
    y=merged_pred_df['price'],
    mode='lines',
    name='Pred Price'
))

# Set the x-axis label
fig.update_xaxes(title_text='Date')

# Set the y-axis label
fig.update_yaxes(title_text='Price')

# Set the title of the plot
fig.update_layout(title='Merged DataFrame')
# Set the y-axis scale to log
fig.update_yaxes(type='log')
# Show the plot

fig.update_layout(
    width=900,  # Set the width of the plot
    height=900  # Set the height of the plot
)



fig.show()

In [12]:
import plotly.graph_objects as go

# Create the plot
fig = go.Figure()

# Add the price_x data
fig.add_trace(go.Scatter(
    x=merged_pred_df['log10 index'],
    y=merged_pred_df['price_x'],
    mode='lines',
    name='Actual Price'
))

# Add the price_y data
fig.add_trace(go.Scatter(
    x=merged_pred_df['log10 index'],
    y=merged_pred_df['price'],
    mode='lines',
    name='Pred Price'
))

# Set the x-axis label
fig.update_xaxes(title_text='Date')

# Set the y-axis label
fig.update_yaxes(title_text='Price')

# Set the title of the plot
fig.update_layout(title='Merged DataFrame')
# Set the y-axis scale to log
fig.update_yaxes(type='log')
# Show the plot

fig.update_layout(
    width=900,  # Set the width of the plot
    height=900  # Set the height of the plot
)



fig.show()

In [ ]:
import plotly.graph_objects as go

# Create the plot
fig = go.Figure()

# Add the random y-axis values
fig.add_trace(go.Scatter(
    x=date_range,
    y=random_numbers,
    mode='markers',
    name='Random Numbers'
))

# Set the x-axis to log scale
fig.update_xaxes(type='log', title_text='Date (Log Scale)')

# Set the y-axis label
fig.update_yaxes(title_text='Random Numbers')

# Set the title of the plot
fig.update_layout(title='Dates from 2010 to 2030 with Random Numbers')

# Show the plot
fig.show()